# HomeWork

In [1]:
import tensorflow as tf
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import numpy as np
from os import listdir,path,mkdir
import random
import shutil
from PIL import Image

2023-06-15 09:45:18.680731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Variables Declare

In [2]:
img_size=(128,128)
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
trainDataPath = '/home/train'
testDataPath = '/home/test'
validationDataPath = '/home/validation'
batchSize = 32
classMode = 'categorical'

# Load the data

In [3]:
def loadData(type):
    if type == 'train':
        trainData = generator.flow_from_directory(
            trainDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode,
        )
        return trainData
    elif type == 'test':
        testData = generator.flow_from_directory(
            testDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode
        )
        return testData



# Define the Model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), # Layer 1
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # Layer 2
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'), # Layer 3
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(), # Full c. layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

2023-06-15 09:45:25.175676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-15 09:45:25.196933: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-15 09:45:25.197575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-15 09:45:25.200646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-15 09:45:25.201314: I tensorflow/compile

# Compile the model

In [5]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
class_labels = ['blacklinewide','bulge','dent','edge','fold','hole','insect','sliver','weldpoint','whitelinewide']
def move(source, dest, split_rate):
    source_files = listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(path.join(source, str(source_files[each_index])), path.join(
                dest, str(source_files[each_index])))
        
def moveAll(source, dest):
    for label in class_labels:
        move(path.join(source, label),
             path.join(dest, label),1.0)

# Kfold

In [7]:
trainData = loadData('train')

Found 1000 images belonging to 10 classes.


In [8]:
moveAll(validationDataPath,trainDataPath)
results=[]
kf = KFold(n_splits=4)
foldIndex = 0
for train_index, val_index in kf.split(trainData.filenames):
    train_files = [trainData.filenames[i] for i in train_index]
    val_files = [trainData.filenames[i] for i in val_index]
    foldIndex+=1
    print("-----Fold:%d-----"%foldIndex)
    for i in val_files:
        shutil.move(path.join(trainDataPath,i),path.join(validationDataPath,i))

    
    train_generator_subset = generator.flow_from_directory(
        trainDataPath,
        target_size=img_size,
        batch_size=batchSize,
        class_mode=classMode,
    )

    val_generator_subset = generator.flow_from_directory(
        trainDataPath,
        target_size=img_size,
        batch_size=batchSize,
        class_mode=classMode,
    )

    model.fit(
        train_generator_subset,
        validation_data=val_generator_subset,
        batch_size=batchSize,
        epochs=10,
        verbose=0
    )
    loss,acc = model.evaluate(val_generator_subset,batch_size=batchSize,verbose=0)
    results.append(acc)
    moveAll(validationDataPath,trainDataPath)

-----Fold:0-----
Found 750 images belonging to 10 classes.
Found 750 images belonging to 10 classes.


2023-06-15 09:45:30.792423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:45:31.699765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-06-15 09:45:33.971214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-15 09:45:35.695171: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f3ce400def0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-15 09:45:35.695245: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor de

-----Fold:0-----
Found 750 images belonging to 10 classes.
Found 750 images belonging to 10 classes.


2023-06-15 09:47:05.252972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:47:10.017993: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:48:22.697719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


-----Fold:0-----
Found 750 images belonging to 10 classes.
Found 750 images belonging to 10 classes.


2023-06-15 09:48:29.723603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:48:33.964408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:49:48.829898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


-----Fold:0-----
Found 750 images belonging to 10 classes.
Found 750 images belonging to 10 classes.


2023-06-15 09:50:04.579495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:50:09.586380: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-15 09:51:21.657444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [10]:
index = 0
for value in results:
    index+=1
    print("Fold %d : %.3f" %(index,value))

Fold 1 : 0.703
Fold 2 : 0.737
Fold 3 : 0.872
Fold 4 : 0.857


In [11]:
loss,acc = model.evaluate(loadData("test"),verbose=0,batch_size=batchSize)
print("Test Accuracy : %3.3f" %(acc))

Found 500 images belonging to 10 classes.


2023-06-15 09:51:27.454470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test Accuracy : 0.674


In [12]:
# model.save(filepath='/home/result.h5')
